## 테이블 데이터 표현하기

In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [2]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [3]:
import csv
wine_path = '/content/drive/MyDrive/deep_learning_with_pytorch/data/p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [4]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list
#데이터 확인

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [6]:
#넘파이 배열을 파이토치 텐서로 변환.
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype
#이제 맛 점수가 기록된 마지막 열까지 들어간 부동소수점 torch.Tensor를 만들었다.

(torch.Size([4898, 12]), torch.float32)

In [7]:
data = wineq[:, :-1] #모든 행과 마지막을 제외한 모든 열을 선택
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [8]:
target = wineq[:, -1] #모든 행과 마지막 열을 선택
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [9]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

## 원핫 인코딩


scatter_ 메소드를 사용한다.

In [10]:
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

#각 행에서 대상이 되는 레이블의 인덱스를 취한 다음 열 인덱스 값으로 사용해서 해당 항목의 값을 1.0으로 지정한다.

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [11]:
#target_onehot이 2차원(4898*10)이므로 unsqueeze를 사용해 target에 추가 차원을 만들어준다.
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

In [12]:
#각 열의 평균과 표준 편차를 구하자.
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.85e+00, 2.78e-01, 3.34e-01, 6.39e+00, 4.58e-02, 3.53e+01,
        1.38e+02, 9.94e-01, 3.19e+00, 4.90e-01, 1.05e+01])

In [13]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
        1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00])

이 경우 dim=0은 차원 0에 대해 축소연산이 수행됨을 지정한다. 이제 학습 과정에 도움이 되도록 열의 값에서 평균을 빼고 표준편차로 나눠 정규화를 진행할 수 있다.

In [14]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.72e-01, -8.18e-02,  ..., -3.49e-01, -1.39e+00],
        [-6.57e-01,  2.16e-01,  ...,  1.34e-03, -8.24e-01],
        ...,
        [-1.61e+00,  1.17e-01,  ..., -9.63e-01,  1.86e+00],
        [-1.01e+00, -6.77e-01,  ..., -1.49e+00,  1.04e+00]])

## 임계값으로 찾기

대충 눈으로 데이터를 직접 들여다보고 좋은 와인과 나쁜 와인을 구별할 쉬운 방법이 있는지 살펴보자. 먼저 점수가 3 이하인 열을 target에서 걸러보자.

In [15]:
bad_indexs = target <= 3
bad_indexs.shape, bad_indexs.dtype, bad_indexs.sum()

#20개

(torch.Size([4898]), torch.bool, tensor(20))

In [16]:
bad_data = data[bad_indexs]
bad_data.shape

torch.Size([20, 11])

In [17]:
#이런 식으로 와인 정보를 좋음, 보통, 나쁨 카테고리로 나눌 수 있다.
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)] # <1>
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


일단 나쁜 와인은 sulfur dioxide 이산화황 성분이 다른 경우보다 높은 듯하다.

대강의 평가 기준으로 이산화황 총량 total sulfur dioxide을 임계값으로 사용할 수 있을 것 같다.

이산화황 총량에서 앞서 계산해 놓은 중앙점보다 낮은 인덱스만 가져와보자.

In [18]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

이 임계값으로는 전체 와인 중 절반 이상을 높은 품질로 구별할 수 있을 것 같다. 이제 실제로 좋은 와인의 인덱스를 뽑아보자.

In [19]:
actual_indexes = target>5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

실제 순위와 우리가 예측한 순서가 얼마나 잘 맞는지 확인할 필요가 있다. 예측한 순위와 실제 품질 순위에 논리 and 연산을 수행해서 얻은 교집합으로 평가해보자.

In [20]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)